In [9]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer

In [10]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [11]:
def feature_inputer(
    df:pd.DataFrame,
    to_mode = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],
    to_median = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
) -> pd.DataFrame:
    for col in to_mode:
        mode_inputer = SimpleImputer(strategy='most_frequent')
        df[col] = mode_inputer.fit_transform(df[[col]])

    
    for col in to_median:
        inputer = SimpleImputer(strategy='median')
        df[col] = inputer.fit_transform(df[[col]])
    return df

def dtype_memory_reducer(df: pd.DataFrame) -> pd.DataFrame:
    df['CryoSleep'] = df['CryoSleep'].astype(bool)
    df['VIP'] = df['VIP'].astype(bool)

    df['HomePlanet'] = df['HomePlanet'].astype('category')
    df['Destination'] = df['Destination'].astype('category')
    return df

def outliers_to_log(df:pd.DataFrame) -> pd.DataFrame:
    to_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in to_log:
        df[col] = np.log(df[col] + 1)
    return df

def cabin_inputer(df:pd.DataFrame) -> pd.DataFrame:
    cabin_features = df['Cabin'].str.split("/",expand=True)[[0,2]].rename(columns={0:'Deck',2:'side'})
    df = pd.concat([df,cabin_features],axis=1)
    df['Deck'] = df['Deck'].fillna('G').replace("T","G")
    df = df.drop(['side','Cabin'],axis=1)
    return df

def vip_knn_input(df: pd.DataFrame) -> pd.DataFrame:
    inputer = KNNImputer(n_neighbors=5)
    df['VIP'] = inputer.fit_transform(df[['VIP','RoomService']])[:,0]
    return df

def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = cabin_inputer(df)
    df = feature_inputer(df)
    df = vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = outliers_to_log(df)
    df = dtype_memory_reducer(df)
    return df

def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

In [13]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)

In [14]:
df_train.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,,
0001_01,False,39.0,False,0.00000,0.000000,0.000000,0.000000,0.000000,False,True,1,0,0,1,1,0,0,0,0,0
0002_01,False,24.0,False,4.70048,2.302585,3.258097,6.309918,3.806662,True,False,0,0,0,1,0,0,0,0,1,0


In [15]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

## Cross Validation

In [17]:
from sklearn.model_selection import cross_val_score, KFold

def model_evaluation(model,x_train,y_train):
    model.fit(x_train, y_train)
    scores = cross_val_score(model, x_train, y_train, cv=KFold(5, shuffle=True, random_state=42), scoring = "accuracy")
    return {
        'model':model,
        'score_avg':np.mean(scores),
        'score_std':np.std(scores)
    }

In [18]:
models = [
    LogisticRegression(max_iter=1000),
    DecisionTreeClassifier(min_samples_leaf=30),
    RandomForestClassifier(min_samples_leaf=20,random_state=42)
]

#grad_boost = GradientBoostingClassifier(min_samples_leaf=20, random_state=42)

In [19]:
df_eva = [model_evaluation(model,X_train,y_train) for model in models]
pd.DataFrame(df_eva)

,model,score_avg,score_std
0,LogisticRegression(max_iter=1000),0.771312,0.008187
1,DecisionTreeClassifier(min_samples_leaf=30),0.789717,0.009159
2,"(DecisionTreeClassifier(max_features='sqrt', m...",0.796848,0.004121


# Submission

In [20]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,
0013_01,True,27.0,False,0.0,0.000000,0.0,0.00000,0.0,True,0,0,0,1,0,0,0,0,0,1
0018_01,False,19.0,False,0.0,2.302585,0.0,7.94591,0.0,False,0,0,0,1,0,0,0,0,1,0


In [21]:
y_test = df_eva[2]['model'].predict(X_test)

In [22]:
submission = pd.Series(y_test, index=[X_test.index]).to_frame('Transported')
submission.to_csv('data/submission_rnforest_2022-10-30.csv')